<a href="https://colab.research.google.com/github/eisbetterthanpi/JEPA/blob/main/I_JEPA_down.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title data
import torch
import torchvision
import torchvision.transforms as transforms
# transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
transform = transforms.Compose([transforms.ToTensor(),])
# transform = transforms.Compose([transforms.RandomResizedCrop((32,32), scale=(.3,1.)), transforms.ToTensor(), transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

# CIFAR10: 60000 32x32 color images in 10 classes, with 6000 images per class
train_data = torchvision.datasets.CIFAR10(root='data', train=True, download=True, transform=transform)
test_data = torchvision.datasets.CIFAR10(root='data', train=False, download=True, transform=transform)
batch_size = 64 # 4
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

import numpy as np
import matplotlib.pyplot as plt
def imshow(img):
    # img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# dataiter = iter(train_loader) # get some random training images
# images, labels = next(dataiter)
# print(images.shape) # [batch, 3, 32, 32]
# imshow(torchvision.utils.make_grid(images))


## vit

In [ ]:
# @title RoPE
import torch
import torch.nn as nn
device = 'cuda' if torch.cuda.is_available() else 'cpu'

def RoPE(dim, seq_len=512, top=torch.pi, base=1000):
    theta = top / (base ** (torch.arange(0, dim, step=2) / dim))
    pos = torch.arange(seq_len).unsqueeze(-1)
    angles = (pos * theta)[None,...,None] # [seq_len, 1] * [dim//2] -> [1, seq_len, dim//2, 1]
    rot_emb = torch.cat([torch.sin(angles), torch.cos(angles)], dim=-1).flatten(-2).to(device) # [1, seq_len, dim//2, 2] -> [1, seq_len, dim]
    return rot_emb


# class RoPE(nn.Module): # Rotary Positional Embeddings
#     def __init__(self, dim, seq_len=512, top=torch.pi, base=1000):
#         super().__init__()
#         self.dim, self.base = dim, base
#         theta = top / (base ** (torch.arange(0, dim, step=2) / dim))
#         pos = torch.arange(seq_len).unsqueeze(-1)
#         angles = (pos * theta)[None,None,...,None] # [seq_len, 1] * [dim//2] -> [1, 1, seq_len, dim//2, 1]
#         self.rot_emb = torch.cat([torch.sin(angles), torch.cos(angles)], dim=-1).flatten(-2).to(device) # [1, 1, seq_len, dim//2, 2] -> [1, 1, seq_len, dim]

#     def forward(self, x): # [b,h,t,d]
#         seq_len = x.size(-2)
#         if self.rot_emb.shape[0] < seq_len: self.__init__(self.dim, seq_len, self.base)
#         return x * self.rot_emb#[:,:,:seq_len]


# @title proper rope
import math
import torch
import torch.nn as nn
device = 'cuda' if torch.cuda.is_available() else 'cpu'

class RoPE(nn.Module):
    def __init__(self, dim, seq_len=512, top=torch.pi, base=1000):
    # def __init__(self, dim, seq_len=512, min_freq=1, max_freq=400, n_zero_freqs=0):
        super().__init__()
        speed = top / (base ** (torch.arange(0, dim, step=2) / dim)) # [dim//2]
        pos = torch.arange(seq_len).unsqueeze(-1) # [t,1]
        # speed = torch.cat([torch.zeros(n_zero_freqs), min_freq * (max_freq/min_freq) ** torch.linspace(0,1,dim//2-n_zero_freqs)]) # [dim//2]
        # pos = torch.linspace(0, 1, seq_len).unsqueeze(-1) # [t,1]

        theta = (speed*pos) # [t,1]*[dim//2]=[t,d//2]
        self.theta = theta
        cos, sin = torch.cos(theta), torch.sin(theta)
        self.affine = torch.stack([cos, -sin, sin, cos], dim=-1).unflatten(-1,(2,2)).to(device) # [t,d//2,4]-> [1,1,t,d//2,2,2]

    def forward(self, x): # [b,h,t,d]
        return (self.affine @ x.unflatten(-1, (-1,2)).unsqueeze(-1)).flatten(-3) # [1,1,t,d//2,2,2] @ [b,h,t,d//2,2,1] = [b,h,t,d]

dim=64
n_heads=4
seq_len=64
rope = RoPE(dim, seq_len, top=torch.pi, base=100)
# rope = RoPE(dim, seq_len, min_freq=1, max_freq=200, n_zero_freqs=0)

x = torch.rand(2, n_heads, seq_len, dim, device=device)
out = rope(x)
print(out.shape)

theta = rope.theta # [t,d//2]
sim = torch.cos(theta-theta[0].unsqueeze(0)).T
# print(sim.shape)

import numpy as np
import matplotlib.pyplot as plt
def imshow(img):
    npimg = img.numpy()
    print(npimg.shape)
    plt.rcParams["figure.figsize"] = (8,8)
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

import torchvision
imshow(torchvision.utils.make_grid(sim, nrow=dim//2))



class RotEmb(nn.Module): # Rotary Positional Embeddings
    def __init__(self, dim, top=torch.pi, base=1000):
        super().__init__()
        self.theta = top / (base ** (torch.arange(0, dim, step=2, device=device) / dim))
        # self.theta = top / (base ** torch.linspace(0, 1, dim//2, device=device))

    def forward(self, pos): # [batch] in [0,1]
        angles = (pos.unsqueeze(-1) * self.theta).unsqueeze(-1) # [seq_len, 1] * [dim // 2] -> [seq_len, dim // 2, 1]
        rot_emb = torch.cat([torch.sin(angles), torch.cos(angles)], dim=-1) # [seq_len, dim // 2, 2]
        return rot_emb.flatten(-2) # [seq_len, dim]

# emb = RotEmb(dim, top=torch.pi, base=1000)

# theta = emb.theta
# print(theta)
# pos = torch.arange(0,200,1)
# angles = (pos.unsqueeze(-1) * theta).T # [b,t]
# sim = torch.cos(angles-angles[:,0].unsqueeze(-1))
# print(sim.shape)

# imshow(torchvision.utils.make_grid(sim, nrow=dim//2))



In [ ]:
# @title RoPE pos
import torch
import torch.nn as nn
device = 'cuda' if torch.cuda.is_available() else 'cpu'

class RoPE(nn.Module):
    def __init__(self, dim, seq_len=512, top=torch.pi, base=10000):
    # def __init__(self, dim, seq_len=512, min_freq=1, max_freq=400, n_zero_freqs=0):
        super().__init__()
        self.dim, self.top, self.base = dim, top, base
        speed = top / (base ** (torch.arange(0, dim, step=2) / dim)) # [dim//2]
        pos = torch.arange(seq_len).unsqueeze(-1) # [t,1]
        # speed = torch.cat([torch.zeros(n_zero_freqs), min_freq * (max_freq/min_freq) ** torch.linspace(0,1,dim//2-n_zero_freqs)]) # [dim//2]
        # pos = torch.linspace(0, 1, seq_len).unsqueeze(-1) # [t,1]
        theta = (speed*pos) # [t,1]*[dim//2]=[t,d//2]
        self.theta = theta
        cos, sin = torch.cos(theta), torch.sin(theta)
        self.affine = torch.stack([cos, -sin, sin, cos], dim=-1).unflatten(-1,(2,2)).to(device)#[None,None,...] # [t,d//2,4]->[t,d//2,2,2] # [1,1,t,d//2,2,2]


        # angles = theta[None,...,None] # [seq_len, 1] * [dim//2] -> [1, 1, seq_len, dim//2, 1]
        # self.rot_emb = torch.cat([torch.sin(angles), torch.cos(angles)], dim=-1).flatten(-2).to(device) # [1, 1, seq_len, dim//2, 2] -> [1, 1, seq_len, dim]
        # # self.rot_emb = torch.cat([sin, cos], dim=-1).flatten(-2).to(device) # [1, 1, seq_len, dim//2, 2] -> [1,1,seq_len,dim]
        # print('self.rot_emb', self.rot_emb.shape)

    def forward(self, x, ind=None): # [b,h,t,d], [b,t]
        # seq_len = x.size(-2)
        b,_,seq_len,_ = x.shape
        if ind!=None: seq_len = max(seq_len, ind.max()+1)
        if self.affine.shape[0] < seq_len: self.__init__(self.dim, seq_len, self.top, self.base)
        if ind!=None:
            # print("if affine, x",self.affine.shape, x.shape)
            return (self.affine.unsqueeze(0).expand(b,-1,-1,-1,-1)[torch.arange(b, device=device).unsqueeze(-1), ind].unsqueeze(1) @ x.unflatten(-1, (-1,2)).unsqueeze(-1)).flatten(-3) # @ [b,h,t,d_head//2,2,1]
        else:
            # print("else affine, x",self.affine.shape, x.shape) # [64, 4, 2, 2], [64, 8, 10, 8]
            return (self.affine[None,None,...] @ x.unflatten(-1, (-1,2)).unsqueeze(-1)).flatten(-3) # @ [b,h,t,d_head//2,2,1]


        # # if self.rot_emb.shape[0] < seq_len: self.__init__(self.dim, seq_len, self.base)
        # if ind==None:
        #     # print("if rot, x",self.rot_emb.shape, x.shape)
        #     return x * self.rot_emb.unsqueeze(1)#[None,None,...]
        # else:
        #     return x * (self.rot_emb.expand(b,-1,-1)[torch.arange(b, device=device).unsqueeze(-1), ind].unsqueeze(1))



dim=64
seq_len=56
rope = RoPE(dim, seq_len, top=torch.pi, base=100)
# rope = RoPE(dim, seq_len, min_freq=1, max_freq=200, n_zero_freqs=0)

batch=2
t=50
x = torch.rand(batch, 4, seq_len, dim, device=device)
out = rope(x)
print("out1", out.shape)
x = torch.rand(batch, 4, t, dim, device=device)
pos = torch.randint(0,seq_len,(batch,t), device=device)
out = rope(x, pos)
print("out2", out.shape)

# theta = rope.theta # [t,d//2]
# sim = torch.cos(theta-theta[0].unsqueeze(0)).T
# # print(sim.shape)

# import numpy as np
# import matplotlib.pyplot as plt
# def imshow(img):
#     npimg = img.numpy()
#     print(npimg.shape)
#     plt.rcParams["figure.figsize"] = (8,8)
#     plt.imshow(np.transpose(npimg, (1, 2, 0)))
#     plt.show()

# import torchvision
# imshow(torchvision.utils.make_grid(sim, nrow=dim//2))



In [ ]:
# @title test gather

dim=64
seq_len=56
batch=2
h=8
t=50
x = torch.rand(1, seq_len, dim, device=device)
pos = torch.randint(0,seq_len,(batch,t), device=device)
# out = x[torch.arange(batch).unsqueeze(-1), pos]
print(out.shape)



# x = torch.rand(1, 1, seq_len, dim, device=device)
# pos = torch.randint(0,seq_len,(batch,t), device=device)

# out = x.expand(batch,-1,-1)[torch.arange(batch)[None,None,:], torch.arange(batch)[None,:,None], pos]
out = x.expand(batch,-1,-1)[torch.arange(batch).unsqueeze(-1), pos]
# out = src.expand_as(src.expand(idx.shape[0], idx.shape[1],-1,-1)).gather(2, idx.unsqueeze(-1).expand(-1,-1,-1,src.size(-1)))



import torch
import time
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# device='cpu'
b = 128
n = 256
t=120
d=64
src = torch.randn(1, n, d, device=device)
idx = torch.randint(0, n, (b,t), device=device)

# print(src[torch.arange(b, device=device), idx].shape)
# print(src.gather(1, idx.unsqueeze(1)).shape)
# print(torch.take_along_dim(src, idx.unsqueeze(1), dim=1).shape)

# out = torch.take_along_dim(src.expand(b,-1,-1), idx.unsqueeze(-1).expand(-1,-1,d), dim=1)
# print(out.shape)
# # out = src.squeeze(0).index_select(0, idx.reshape(-1)).reshape(b, t, d)
# out = src.index_select(0, idx.reshape(-1)).reshape(b, t, d)
# print(out.shape)
# out = src.expand(b, -1, -1).gather(1, idx.unsqueeze(-1).expand(-1, -1, d))
# print(out.shape)

# src: (1, n, d)
# idx: (b, t)

out = src.expand(b,-1,-1).gather(1, idx.unsqueeze(-1).expand(-1, -1, src.size(-1)))
print(out.shape)
out = src.expand(b,-1,-1)[torch.arange(b, device=device).unsqueeze(-1), idx]
print(out.shape)

# %timeit out = src.expand(b,-1,-1).gather(1, idx.unsqueeze(-1).expand(-1, -1, src.size(-1))) # 2.53 ms
%timeit out = src.expand(b,-1,-1)[torch.arange(b, device=device).unsqueeze(-1), idx] # 1.39 ms


In [ ]:
# @title GoldenGateRoPE2d
import math
import torch
import torch.nn as nn
device = 'cuda' if torch.cuda.is_available() else 'cpu'

class GoldenGateRoPE2d(nn.Module): # jerryxio.ng/posts/nd-rope
    def __init__(self, image_size, n_heads, n_freqs, min_freq=.8, max_freq=10, n_zero_freqs=0):
        super().__init__()
        intv = math.pi * (math.sqrt(5)-1)/2 # mod pi instead of 2pi # pi*(sqrt5+-1)/2 ; + and - are equivalent bec mod pi
        # intv = math.pi * (math.sqrt(5)-1) # https://en.wikipedia.org/wiki/Golden_angle
        speed = torch.cat([torch.zeros(n_zero_freqs), min_freq * (max_freq/min_freq) ** torch.linspace(0,1,n_freqs-n_zero_freqs)]).unsqueeze(-1) # [n_freqs,1] # og
        angle = torch.arange(n_heads*n_freqs).reshape(n_heads, n_freqs) * intv # [n_heads, n_freqs]
        direction = torch.stack([torch.cos(angle), torch.sin(angle)], dim=-1) # [n_heads, n_freqs, 2]
        h, w = image_size
        xlim, ylim = w/h, h/w
        y, x = torch.meshgrid(torch.linspace(-ylim, ylim, h), torch.linspace(-xlim, xlim, w), indexing="ij") # [h,w], y:row_num, x:col_num
        pos = torch.stack([x, y], dim=-1).reshape(-1,1,1,2) # [h*w,1,1,2] cartesian coords
        theta = (speed*direction*pos).sum(dim=-1) # [t,n_heads,n_freqs,2]->[t,n_heads,d_head]
        self.theta = theta
        cos, sin = torch.cos(theta), torch.sin(theta)
        self.affine = torch.stack([cos, -sin, sin, cos], dim=-1).transpose(0,1).reshape(1,n_heads,h*w,n_freqs,2,2).to(device) # [t,n_heads,n_freqs,4]->[1,n_heads,t,n_freqs,2,2]

    def forward(self, x): # [b,h,t,d]
        return (self.affine @ x.unflatten(-1, (-1,2)).unsqueeze(-1)).flatten(-3) # @ [b,h,t,d_head//2,2,1]

# /2 better
# speed [n_freqs,1] # og best
# w/h best
# rope < ggrope < learned

# image_size=(8,8)
image_size=(20,30)
# image_size=(90,120)
n_heads=4
n_freqs=6
ggrope = GoldenGateRoPE2d(image_size, n_heads, n_freqs)

# x = torch.rand(2, *image_size, n_heads, n_freqs*2)
x = torch.rand(2, n_heads, image_size[0]*image_size[1], n_freqs*2)
out = ggrope(x)
print(out.shape)
# # print(out[0])
# theta = ggrope.theta.flatten(-2).permute(2,0,1).unsqueeze(1) # [t,n_heads,d_head][b,1,h,w]
theta = ggrope.theta.flatten(-2).T.reshape(n_heads*n_freqs, 1, *image_size) # [t,n_heads,d_head]->[d,1,h,w]
cy, cx = image_size[0]//2, image_size[1]//2
sim = torch.cos(theta-theta[...,cy,cx][...,None,None]) # [b,1,h,w]
# sim = sim.unflatten(0, (n_heads, n_freqs)).mean(1)
# print(sim.shape)

import numpy as np
import matplotlib.pyplot as plt
def imshow(img):
    npimg = img.numpy()
    print(npimg.shape)
    plt.rcParams["figure.figsize"] = (8,8)
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
# bhwc

import torchvision
imshow(torchvision.utils.make_grid(sim, nrow=n_freqs))



In [ ]:
# @title AttentionBlock
import torch
import torch.nn as nn
device = 'cuda' if torch.cuda.is_available() else 'cpu'

def zero_module(module):
    for p in module.parameters():
        p.detach().zero_()
    return module

class SelfAttn(nn.Module):
    def __init__(self, dim, n_heads):
        super().__init__()
        self.dim, self.n_heads = dim, n_heads
        d_head = dim//n_heads
        self.rope = RoPE(d_head, seq_len=64, top=torch.pi, base=1000)
        # self.rope = RoPE(d_head, seq_len=512, base=1000)
        # self.rope = RoPE2D(d_head, h=64, w=64, base=100)
        # self.rope[0] = 1 # id for smry h
        self.qkv = nn.Linear(dim, dim*3, bias=False)
        # self.lin = nn.Linear(dim, dim)
        self.lin = zero_module(nn.Linear(dim, dim))

        self.scale = d_head**-.5

    # def forward(self, x): # [b,t,d]
    def forward(self, x, pos=None): # [b,t,d]
        q,k,v = self.qkv(x).unflatten(-1, (self.n_heads,-1)).transpose(1,2).chunk(3, dim=-1) # [b, t, n_heads, d_head] -> [b, n_heads, t, d_head]
        # q, k = self.rope(q), self.rope(k)
        if pos != None:
            # rope = self.rope[pos]
            # q, k = q*rope, k*rope
            q, k = self.rope(q, pos), self.rope(k, pos)

        # x = F.scaled_dot_product_attention(q,k,v, attn_mask=None) # https://pytorch.org/docs/stable/generated/torch.nn.functional.scaled_dot_product_attention.html
        q, k = q.softmax(dim=-1)*self.scale, k.softmax(dim=-2)
        context = k.transpose(-2,-1) @ v # [batch, n_heads, d_head, d_head]
        x = q @ context # [b, n_heads, t, d_head]
        # # print('SelfAttn', x.shape)
        x = x.transpose(1,2).flatten(2)
        return self.lin(x)

class AttentionBlock(nn.Module):
    def __init__(self, d_model, n_heads, mult=4, drop=0.):
        super().__init__()
        self.d_model = d_model
        # self.norm = nn.RMSNorm(d_model, elementwise_affine=False) # LayerNorm RMSNorm
        self.norm1, self.norm2 = nn.LayerNorm(d_model), nn.LayerNorm(d_model)
        self.drop = nn.Dropout(drop)
        self.attn = SelfAttn(d_model, n_heads)
        act = nn.GELU() # ReLU GELU
        ff_dim=d_model*mult
        self.ff = nn.Sequential(
            nn.RMSNorm(d_model), act, nn.Dropout(drop), nn.Linear(d_model, ff_dim),
            # nn.RMSNorm(ff_dim), act, nn.Dropout(drop), nn.Linear(ff_dim, d_model)
            nn.RMSNorm(ff_dim), act, nn.Dropout(drop), zero_module(nn.Linear(ff_dim, d_model))
            # nn.RMSNorm(d_model), nn.Dropout(drop), nn.Linear(d_model, ff_dim), act,
            # nn.RMSNorm(ff_dim), nn.Dropout(drop), nn.Linear(ff_dim, d_model)
        )

    # def forward(self, x): # [b,t,d]
    def forward(self, x, pos=None): # [b,t,d]
        # print('attnblk fwd',x.shape)
        # x = x + self.drop(self.attn(self.norm1(x)))
        x = x + self.drop(self.attn(self.norm1(x), pos))
        x = x + self.ff(x)
        # x = x + self.drop(self.ff(self.norm2(x)))
        return x


class ViT(nn.Module):
    # def __init__(self, in_dim, d_model, out_dim=None, nhead=8, d_hid=None, nlayers=1, dropout = 0.):
    def __init__(self, patch_size, in_dim, d_model, out_dim=None, n_heads=4, nlayers=1, drop=0.):
        super().__init__()
        patch_size=2
        self.embed = nn.Sequential(
            # nn.Conv2d(in_dim, d_model, patch_size, patch_size), # like patch
            # nn.Conv2d(in_dim, d_model, 7, 1, 7//2, bias=False)
            # nn.Conv2d(in_dim, d_model, 7, 2, 7//2, bias=False), nn.MaxPool2d(3,2,1)
            nn.Conv2d(in_dim, d_model, 7, 2, 7//2, bias=False), nn.BatchNorm2d(d_model), nn.ReLU(),
            nn.Conv2d(d_model, d_model, 3, 2, 3//2, bias=False)
            )
        # self.embed.requires_grad=False
        # self.pos_enc = RotEmb(d_model, top=1, base=10000)
        self.pos_emb = nn.Parameter(torch.randn(1, 32*32, d_model)*.02)
        # self.pos_emb = nn.Parameter(RoPE2D(dim=d_model, h=8, w=8, base=1000), requires_grad=False)

        # self.pos_emb = nn.Parameter(RoPE(d_model, seq_len=(32//patch_size)**2, base=10000), requires_grad=False)
        self.transformer = nn.Sequential(*[AttentionBlock(d_model, d_model, n_heads) for _ in range(nlayers)])
        self.transformer = Seq(*[AttentionBlock(d_model, d_model, n_heads) for _ in range(nlayers)])
        self.norm = nn.RMSNorm(d_model) # LayerNorm RMSNorm
        self.lin = nn.Linear(d_model, out_dim) if out_dim and out_dim != d_model else None

    def forward(self, x, context_indices=None): # [batch, num_context_toks, 3], [batch, num_context_toks] # True will be ignored by the attention # https://pytorch.org/docs/stable/generated/torch.nn.Transformer.html
        x = self.embed(x).flatten(2).transpose(1,2) # [b,c,h,w]->[b,h*w,c] # [batch, seq_len, d_model] or [batch, num_context_toks, d_model]
        # x = self.pos_enc(x)
        # print("TransformerModel",x.shape, self.pos_emb.shape)
        # x = x + self.pos_emb[:,:x.shape[1]]
        # if context_indices != None: x = x[torch.arange(x.shape[0]).unsqueeze(-1), context_indices] # [batch, num_context_toks, d_model]
        # x = self.transformer(x)

        x = self.transformer(x, context_indices)
        out = self.norm(x)
        if self.lin: out = self.lin(out)
        return out


d_model = 64
in_dim = 3
patch_size = 2
# model = ViT(patch_size, in_dim, d_model, n_heads=4, nlayers=4, drop=0.).to(device)
model = ViT(patch_size, in_dim, d_model, n_heads=4, nlayers=1, drop=0.).to(device)
print(sum(p.numel() for p in model.parameters() if p.requires_grad)) # 27584
x = torch.rand((5, in_dim, 32, 32), device=device)
out = model(x)
print(out.shape)




In [ ]:
# @title AttentionBlock pos
import torch
import torch.nn as nn
device = 'cuda' if torch.cuda.is_available() else 'cpu'

def zero_module(module):
    for p in module.parameters():
        p.detach().zero_()
    return module

import inspect
class Seq(nn.Sequential):
    def __init__(self, *args):
        super().__init__(*args)
        for layer in self:
            params = inspect.signature(layer.forward).parameters.keys()
            layer._fwdparams = ','.join(params)

    def forward(self, x, pos=None, masks=None):
        arg_map = {'pos':pos, 'masks':masks}
        for layer in self:
            args = [x]
            # if 'masks' in layer._fwdparams: args.append(masks)
            args.extend(arg_map[p] for p in arg_map if p in layer._fwdparams)
            # print(layer._fwdparams, args)
            x = layer(*args)
        return x

class SelfAttn(nn.Module):
    def __init__(self, dim, n_heads):
        super().__init__()
        self.dim, self.n_heads = dim, n_heads
        d_head = dim//n_heads
        self.rope = RoPE(d_head, seq_len=64, top=torch.pi, base=1000)
        # self.rope = RoPE(d_head, seq_len=512, base=1000)
        # self.rope = RoPE2D(d_head, h=64, w=64, base=100)
        # self.rope[0] = 1 # id for smry h
        self.qkv = nn.Linear(dim, dim*3, bias=False)
        # self.lin = nn.Linear(dim, dim)
        self.lin = zero_module(nn.Linear(dim, dim))
        self.scale = d_head**-.5

    # def forward(self, x): # [b,t,d]
    def forward(self, x, pos=None): # [b,t,d]
        # print('SelfAttn', x.shape)
        q,k,v = self.qkv(x).unflatten(-1, (self.n_heads,-1)).transpose(1,2).chunk(3, dim=-1) # [b, t, n_heads, d_head] -> [b, n_heads, t, d_head]
        if pos==None: q, k = self.rope(q), self.rope(k)
        else: q, k = self.rope(q, pos), self.rope(k, pos)

        x = F.scaled_dot_product_attention(q,k,v, attn_mask=None) # https://pytorch.org/docs/stable/generated/torch.nn.functional.scaled_dot_product_attention.html
        # q, k = q.softmax(dim=-1)*self.scale, k.softmax(dim=-2)
        # context = k.transpose(-2,-1) @ v # [batch, n_heads, d_head, d_head]
        # x = q @ context # [b,n_heads,t,d_head]
        x = x.transpose(1,2).flatten(2)
        return self.lin(x)

class AttentionBlock(nn.Module):
    def __init__(self, d_model, n_heads, mult=4, drop=0.):
        super().__init__()
        self.d_model = d_model
        # self.norm = nn.RMSNorm(d_model, elementwise_affine=False) # LayerNorm RMSNorm
        self.norm1, self.norm2 = nn.LayerNorm(d_model), nn.LayerNorm(d_model)
        self.drop = nn.Dropout(drop)
        self.attn = SelfAttn(d_model, n_heads)
        act = nn.GELU() # ReLU GELU
        ff_dim=d_model*mult
        self.ff = nn.Sequential(
            nn.RMSNorm(d_model), act, nn.Dropout(drop), nn.Linear(d_model, ff_dim),
            # nn.RMSNorm(ff_dim), act, nn.Dropout(drop), nn.Linear(ff_dim, d_model)
            nn.RMSNorm(ff_dim), act, nn.Dropout(drop), zero_module(nn.Linear(ff_dim, d_model))
            # nn.RMSNorm(d_model), nn.Dropout(drop), nn.Linear(d_model, ff_dim), act,
            # nn.RMSNorm(ff_dim), nn.Dropout(drop), nn.Linear(ff_dim, d_model)
        )

    # def forward(self, x): # [b,t,d]
    def forward(self, x, pos=None): # [b,t,d]
        # print('attnblk fwd',x.shape)
        # x = x + self.drop(self.attn(self.norm1(x)))
        x = x + self.drop(self.attn(self.norm1(x), pos))
        x = x + self.ff(x)
        # x = x + self.drop(self.ff(self.norm2(x)))
        return x


class ViT(nn.Module):
    # def __init__(self, in_dim, d_model, out_dim=None, nhead=8, d_hid=None, nlayers=1, dropout = 0.):
    def __init__(self, patch_size, in_dim, d_model, out_dim=None, n_heads=4, nlayers=1, drop=0.):
        super().__init__()
        patch_size=2
        self.embed = nn.Sequential(
            # nn.Conv2d(in_dim, d_model, patch_size, patch_size), # like patch
            # nn.Conv2d(in_dim, d_model, 7, 1, 7//2, bias=False)
            # nn.Conv2d(in_dim, d_model, 7, 2, 7//2, bias=False), nn.MaxPool2d(3,2,1)
            nn.Conv2d(in_dim, d_model, 7, 2, 7//2, bias=False), nn.BatchNorm2d(d_model), nn.ReLU(),
            nn.Conv2d(d_model, d_model, 3, 2, 3//2, bias=False)
            )
        # self.embed.requires_grad=False
        # self.pos_enc = RotEmb(d_model, top=1, base=10000)
        self.pos_emb = nn.Parameter(torch.randn(1, 32*32, d_model)*.02)
        # self.pos_emb = nn.Parameter(RoPE2D(dim=d_model, h=8, w=8, base=1000), requires_grad=False)

        # self.pos_emb = nn.Parameter(RoPE(d_model, seq_len=(32//patch_size)**2, base=10000), requires_grad=False)
        # self.transformer = nn.Sequential(*[AttentionBlock(d_model, d_model, n_heads) for _ in range(nlayers)])
        self.transformer = Seq(*[AttentionBlock(d_model, n_heads) for _ in range(nlayers)])
        self.norm = nn.RMSNorm(d_model) # LayerNorm RMSNorm
        self.lin = nn.Linear(d_model, out_dim) if out_dim and out_dim != d_model else None

    def forward(self, x, context_indices=None): # [batch, num_context_toks, 3], [batch, num_context_toks] # True will be ignored by the attention # https://pytorch.org/docs/stable/generated/torch.nn.Transformer.html
        x = self.embed(x).flatten(2).transpose(1,2) # [b,c,h,w]->[b,h*w,c] # [batch, seq_len, d_model] or [batch, num_context_toks, d_model]
        # x = self.pos_enc(x)
        # print("TransformerModel",x.shape, self.pos_emb.shape)
        # print("TransformerModel",x.shape)
        # x = x + self.pos_emb[:,:x.shape[1]]
        # if context_indices != None: x = x[torch.arange(x.shape[0]).unsqueeze(-1), context_indices] # [batch, num_context_toks, d_model]
        # x = self.transformer(x)


        if context_indices != None:
            # print('vit transformer',x[torch.arange(x.shape[0]).unsqueeze(-1), context_indices].shape, context_indices.shape)
            x = self.transformer(x[torch.arange(x.shape[0]).unsqueeze(-1), context_indices], context_indices)
        else: x = self.transformer(x)

        out = self.norm(x)
        if self.lin: out = self.lin(out)
        return out

d_model = 64
in_dim = 3
patch_size = 2
# model = ViT(patch_size, in_dim, d_model, n_heads=4, nlayers=4, drop=0.).to(device)
model = ViT(patch_size, in_dim, d_model, n_heads=4, nlayers=1, drop=0.).to(device)
print(sum(p.numel() for p in model.parameters() if p.requires_grad)) # 27584
x = torch.rand((5, in_dim, 32, 32), device=device) # [b,c,h,w]
out = model(x)
print(out.shape)


In [ ]:
# @title random_masking
import torch

def random_masking(length, mask_ratio, b=64):
    noise = torch.rand(b, length)
    len_mask = int(length * mask_ratio)
    _, msk_ind = torch.topk(noise, k=len_mask, dim=-1, sorted=False) # val, ind -> [b,len_mask]
    _, keep_ind = torch.topk(noise, k=length-len_mask, largest=False, dim=-1, sorted=False) # val, ind -> [b,len_keep]
    return msk_ind, keep_ind

# msk_ind, keep_ind = random_masking(10, .3, b=2)

# x_ = torch.rand(4, 3, 2)
# print(x_)
# # ids = torch.tensor([0, 2, 1])[None,:,None]
# # ids = torch.tensor([0, 2, 1])[None,:,None].repeat(4,1,2)
# ids = torch.tensor([1, 2, 0])[None,:,None].repeat(4,1,2)
# # o = torch.gather(x_, dim=1, index=ids)
# o = torch.zeros_like(x_).scatter_(dim=1, index=ids, src=x_)
# print(o)


In [ ]:
# @title simplex
# !pip install -q opensimplex
import opensimplex
import numpy as np
import torch

def simplexmask2d(hw=(32,32), ctx_scale=(.85,1.), trg_scale=(.6,.8), B=64, chaos=[1,.5]):
    ix, iy = np.split(np.linspace(0, chaos[0], num=sum(hw)), [hw[0]])
    noise = opensimplex.noise3array(ix, iy, np.random.randint(1e10, size=B)) # [b,h,w]
    noise = torch.from_numpy(noise).flatten(1)
    trunc_normal = torch.fmod(torch.randn(2)*.3,1)/2 + .5
    ctx_mask_scale = torch.rand(1) * (ctx_scale[1] - ctx_scale[0]) + ctx_scale[0] # in (min_s, max_s) # all blocks same size
    trg_mask_scale = torch.rand(1) * (trg_scale[1] - trg_scale[0]) + trg_scale[0]
    # ctx_mask_scale = trunc_normal[0] * (ctx_scale[1] - ctx_scale[0]) + ctx_scale[0] # in (min_s, max_s) # all blocks same size
    # trg_mask_scale = trunc_normal[1] * (trg_scale[1] - trg_scale[0]) + trg_scale[0]
    seq = hw[0]*hw[1]
    ctx_len, trg_len = int(seq*ctx_mask_scale), int(seq*trg_mask_scale)
    val, trg_index = torch.topk(noise, trg_len, dim=1, sorted=False)
    # ctx_len = ctx_len - trg_len
    ctx_len = min(ctx_len, seq-trg_len)

    remove_mask = torch.ones((B,seq), dtype=bool) # [B, S]
    remove_mask.scatter_(1, trg_index, False).flatten()
    ind = torch.arange(seq).unsqueeze(0).repeat(B,1)[remove_mask].reshape(B, -1)

    ix, iy = np.split(np.linspace(0, chaos[1], num=sum(hw)), [hw[0]])
    noise = opensimplex.noise3array(ix, iy, np.random.randint(1e10, size=B)) # [b,h,w]
    noise = torch.from_numpy(noise).flatten(1)[remove_mask].reshape(B, -1)
    val, ctx_ind = torch.topk(noise, ctx_len, dim=1, sorted=False)
    ctx_index = ind[torch.arange(B).unsqueeze(-1), ctx_ind]
    return ctx_index, trg_index

b=16
# ctx_index, trg_index = simplexmask2d(hw=(32,32), ctx_scale=(.85,1), trg_scale=(.7,.8), B=b, chaos=[3,.5])
# ctx_index, trg_index = simplexmask2d(hw=(32,32), ctx_scale=(.85,1), trg_scale=(.7,.8), B=b, chaos=[3,1])
# ctx_index, trg_index = simplexmask2d(hw=(32,32), ctx_scale=(.85,1), trg_scale=(.2,.8), B=b, chaos=[3,1])
ctx_index, trg_index = simplexmask2d(hw=(32,32), ctx_scale=(.05,.2), trg_scale=(.7,.8), B=b, chaos=[3,1])
# ctx_index, trg_index = simplexmask2d(hw=(32,32), ctx_scale=(.85,1), trg_scale=(.5,.5), B=b, chaos=[.01,.5])


import matplotlib.pyplot as plt
def imshow(img):
    npimg = img.numpy()
    plt.rcParams["figure.figsize"] = (8,8)
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
# imshow(mask)

mask = torch.zeros(b ,32*32)
# mask = torch.ones(b ,32*32)*.3
mask[torch.arange(b).unsqueeze(-1), trg_index] = 1
mask[torch.arange(b).unsqueeze(-1), ctx_index] = .5
# print((mask==1).sum(1))
# print((mask==.5).sum(1))
mask = mask.reshape(b,1,32,32)
import torchvision
imshow(torchvision.utils.make_grid(mask, nrow=8))

#
ttc=[]
ttt=[]
def mean(x): return sum(x)/len(x)

for i in range(1000):
    # collated_masks_enc, collated_masks_pred = mask_collator(1)
    # context_indices, trg_indices = torch.stack(collated_masks_enc).squeeze(0), torch.stack(collated_masks_pred).transpose(0,1).flatten(1).unique(dim=1) # [num_msk, b,num_tok]->[b,num_tok] # [64, 65], [64, 32]
    # context_indices, trg_indices = simplexmask2d(hw=(32,32), ctx_scale=(.85,1.), trg_scale=(.7,.8), B=1, chaos=[3,1])
    context_indices, trg_indices = simplexmask2d(hw=(32,32), ctx_scale=(.05,.5), trg_scale=(.2,.8), B=1, chaos=[3,1])
    ttc.append(context_indices.shape[-1])
    ttt.append(trg_indices.shape[-1])

print(mean(ttc), mean(ttt))

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (4,4)
plt.hist(ttc, bins=20, alpha=.5, label='context mask')
plt.hist(ttt, bins=20, alpha=.5, label='target mask')
plt.legend()
plt.show()



In [ ]:
# @title masks
import torch
import numpy as np
import matplotlib.pyplot as plt

# def multiblock(seq, min_s, max_s, M=1):
#     mask_len = torch.rand(1) * (max_s - min_s) + min_s # in (min_s, max_s) # all blocks same size
#     mask_pos = torch.rand(M) * (1 - mask_len) # in (0, 1 - mask_len)
#     mask_len, mask_pos = (mask_len * seq).int(), mask_pos * seq
#     indices = torch.arange(seq).unsqueeze(0) # [1, seq]
#     target_mask = (indices >= mask_pos.unsqueeze(-1)) & (indices < (mask_pos + mask_len).unsqueeze(-1)) # [M, seq]
#     return target_mask


# def multiblock2d(hw=(8,8), scale=(.15,.2), aspect_ratio=(.75,1.5), M=1):
#     mask_aspect = torch.rand(1) * (aspect_ratio[1] - aspect_ratio[0]) + aspect_ratio[0] # in (min_s, max_s) # all blocks same size
#     mask_scale = torch.rand(1) * (scale[1] - scale[0]) + scale[0] # in (min_s, max_s) # all blocks same size
#     h = (mask_scale/mask_aspect)**.5# h*(h*aspect) = scale
#     w = h * mask_aspect
#     h_pos, w_pos = torch.rand(M)*(1-w), torch.rand(M)*(1-h) # in (0, 1 - mask_len)
#     h_len, h_pos = (h*hw[0]).int(), h_pos*hw[0]
#     w_len, w_pos = (w*hw[1]).int(), w_pos*hw[1]
#     h_ind, w_ind = torch.arange(hw[0]).unsqueeze(0), torch.arange(hw[1]).unsqueeze(0) # [1, seq]
#     h_mask = (h_ind>=h_pos.unsqueeze(-1)) & (h_ind<(h_pos+h_len).unsqueeze(-1)) # [M, seq]
#     w_mask = (w_ind>=w_pos.unsqueeze(-1)) & (w_ind<(w_pos+w_len).unsqueeze(-1)) # [M, seq]
#     target_mask = h_mask.unsqueeze(-1) & w_mask.unsqueeze(-2) # [M, seq, seq]
#     return target_mask

# # https://arxiv.org/pdf/2210.07224
# def randpatch(seq, mask_size=8, gamma=0.9): # num patches of seq, mask patch size, masking ratio
#     # mask = torch.rand(seq//mask_size)<gamma
#     length = seq//mask_size
#     g = torch.normal(gamma, std=.1, size=(1,)).clamp(.5,.9)
#     # g = gamma
#     idx = torch.randperm(length)[:int(length*g)]
#     mask = torch.zeros(length, dtype=bool)
#     mask[idx] = True
#     mask = mask.repeat_interleave(mask_size, dim=-1)
#     return mask # [seq] , True -> mask


# import torch
# def apply_masks(x, mask): # [b,t,d], [mask_size] # https://github.com/facebookresearch/ijepa/blob/main/src/masks/utils.py
#     mask_keep = mask.unsqueeze(-1).repeat(x.size(0), 1, x.size(-1)) # [batch,T,dim]
#     return torch.gather(x, dim=1, index=mask_keep) # [batch,mask_size,dim]





# @title ijepa multiblock next
# https://github.com/facebookresearch/ijepa/blob/main/src/masks/multiblock.py
import math
from multiprocessing import Value
import torch

class MaskCollator(object):
    def __init__(self, hw=(224, 224), enc_mask_scale=(.85,1), pred_mask_scale=(.15,.2), aspect_ratio=(.75,1.25),
        nenc=1, npred=2, min_keep=4, allow_overlap=False):
        super().__init__()
        self.height, self.width = hw
        self.enc_mask_scale = enc_mask_scale
        self.pred_mask_scale = pred_mask_scale
        self.aspect_ratio = aspect_ratio
        self.nenc = nenc
        self.npred = npred
        self.min_keep = min_keep  # minimum number of patches to keep
        self.allow_overlap = allow_overlap  # whether to allow overlap b/w enc and pred masks

    def _sample_block_size(self, scale, aspect_ratio_scale):
        _rand = torch.rand(1).item()
        # -- Sample block scale
        min_s, max_s = scale
        mask_scale = min_s + _rand * (max_s - min_s)
        max_keep = int(self.height * self.width * mask_scale) # num patches to keep
        # -- Sample block aspect-ratio
        min_ar, max_ar = aspect_ratio_scale
        aspect_ratio = min_ar + _rand * (max_ar - min_ar)
        # -- Compute block height and width (given scale and aspect-ratio)
        h = int(round(math.sqrt(max_keep * aspect_ratio)))
        w = int(round(math.sqrt(max_keep / aspect_ratio)))
        while h >= self.height: h -= 1 # crop mask to be smaller than img
        while w >= self.width: w -= 1
        return (h, w)

    def _sample_block_mask(self, b_size, acceptable_regions=None):
        h, w = b_size
        def constrain_mask(mask, tries=0):
            """ Helper to restrict given mask to a set of acceptable regions """
            N = max(int(len(acceptable_regions)-tries), 0)
            for k in range(N):
                mask *= acceptable_regions[k]
        # -- Loop to sample masks until we find a valid one
        tries = 0
        timeout = og_timeout = 20
        valid_mask = False
        while not valid_mask:
            # -- Sample block top-left corner
            top = torch.randint(0, self.height - h, (1,))
            left = torch.randint(0, self.width - w, (1,))
            mask = torch.zeros((self.height, self.width), dtype=torch.int32)
            mask[top:top+h, left:left+w] = 1
            # -- Constrain mask to a set of acceptable regions
            if acceptable_regions is not None:
                constrain_mask(mask, tries)
            mask = torch.nonzero(mask.flatten())
            # -- If mask too small try again
            valid_mask = len(mask) > self.min_keep
            if not valid_mask:
                timeout -= 1
                if timeout == 0:
                    tries += 1
                    timeout = og_timeout
        mask = mask.squeeze()
        # --
        mask_complement = torch.ones((self.height, self.width), dtype=torch.int32)
        mask_complement[top:top+h, left:left+w] = 0
        # --
        return mask, mask_complement

    def __call__(self, B):
        '''
        Create encoder and predictor masks when collating imgs into a batch
        # 1. sample enc block (size + location) using seed
        # 2. sample pred block (size) using seed
        # 3. sample several enc block locations for each image (w/o seed)
        # 4. sample several pred block locations for each image (w/o seed)
        # 5. return enc mask and pred mask
        '''
        p_size = self._sample_block_size(scale=self.pred_mask_scale, aspect_ratio_scale=self.aspect_ratio)
        e_size = self._sample_block_size(scale=self.enc_mask_scale, aspect_ratio_scale=(1., 1.))

        collated_masks_pred, collated_masks_enc = [], []
        min_keep_pred = self.height * self.width
        min_keep_enc = self.height * self.width
        for _ in range(B):

            masks_p, masks_C = [], []
            for _ in range(self.npred):
                mask, mask_C = self._sample_block_mask(p_size)
                masks_p.append(mask)
                masks_C.append(mask_C)
                min_keep_pred = min(min_keep_pred, len(mask))
            collated_masks_pred.append(masks_p)

            acceptable_regions = masks_C
            try:
                if self.allow_overlap:
                    acceptable_regions= None
            except Exception as e:
                print(f'Encountered exception in mask-generator {e}')

            masks_e = []
            for _ in range(self.nenc):
                mask, _ = self._sample_block_mask(e_size, acceptable_regions=acceptable_regions)
                masks_e.append(mask)
                min_keep_enc = min(min_keep_enc, len(mask))
            collated_masks_enc.append(masks_e)
        collated_masks_pred = [[cm[:min_keep_pred] for cm in cm_list] for cm_list in collated_masks_pred]
        collated_masks_pred = torch.utils.data.default_collate(collated_masks_pred)
        # --
        collated_masks_enc = [[cm[:min_keep_enc] for cm in cm_list] for cm_list in collated_masks_enc]
        collated_masks_enc = torch.utils.data.default_collate(collated_masks_enc)
        return collated_masks_enc, collated_masks_pred

mask_collator = MaskCollator(hw=(32,32), enc_mask_scale=(.85, 1.), pred_mask_scale=(.15, .2), aspect_ratio=(.75, 1.5),
        nenc=1, npred=4, min_keep=4,
        # allow_overlap=True)
        allow_overlap=False)

b=16
collated_masks_enc, collated_masks_pred = mask_collator(b)
ctx_index, trg_index = torch.stack(collated_masks_enc).squeeze(0), torch.stack(collated_masks_pred).transpose(0,1).flatten(1).unique(dim=1) # [num_msk, b,num_tok]->[b,num_tok] # [64, 65], [64, 32]

# mask = torch.zeros(1 ,32*32)
# mask[:, trg_index[:1]] = 1
# mask[:, ctx_index[:1]] = .5
# mask = mask.reshape(1,32,32)

import numpy as np
import matplotlib.pyplot as plt
def imshow(img):
    npimg = img.numpy()
    plt.rcParams["figure.figsize"] = (8,8)
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
# imshow(mask)

mask = torch.zeros(b ,32*32)
mask[torch.arange(b).unsqueeze(-1), trg_index] = 1
mask[torch.arange(b).unsqueeze(-1), ctx_index] = .5
mask = mask.reshape(b,1,32,32)
import torchvision
imshow(torchvision.utils.make_grid(mask, nrow=8))

# mask_collator = MaskCollator(hw=(1024,1024), enc_mask_scale=(.85, 1.), pred_mask_scale=(.15, .2), aspect_ratio=(.75, 1.5), nenc=1, npred=4, min_keep=4, allow_overlap=False)
# # %timeit collated_masks_enc, collated_masks_pred = mask_collator(64) # 225 ms 1024:4.79 s
# # %timeit ctx_index, trg_index = simplexmask2d(hw=(1024,1024), ctx_scale=(.85,1), trg_scale=(.5,.6), B=b, chaos=.5) # 265 ms ;topk 203 ms 1024:4.27 s


In [ ]:
# @title Transformer Predictor
import torch
import torch.nn as nn
device = 'cuda' if torch.cuda.is_available() else 'cpu'

class TransformerPredictor(nn.Module):
    def __init__(self, in_dim, d_model, out_dim=None, n_heads=4, nlayers=1, drop=0.):
        super().__init__()
        self.embed = nn.Linear(in_dim, d_model)# if in_dim != d_model else None
        # self.pos_enc = RotEmb(d_model, top=1, base=10000)
        self.pos_emb = nn.Parameter(torch.randn(1, 32*32, d_model)*0.02)
        # self.pos_emb = nn.Parameter(RoPE2D(dim=d_model, h=8, w=8, base=1000), requires_grad=False)
        # self.pos_emb = nn.Parameter(RoPE(d_model, seq_len=32*32, base=10000), requires_grad=False)

        # self.transformer = nn.Sequential(*[AttentionBlock(d_model, d_model, n_heads) for _ in range(nlayers)])
        self.transformer = Seq(*[AttentionBlock(d_model, n_heads) for _ in range(nlayers)])

        self.cls = nn.Parameter(torch.randn(1,1,d_model)*0.02) # randn zeros
        out_dim = out_dim or d_model
        self.norm = nn.RMSNorm(d_model) # LayerNorm RMSNorm
        self.lin = nn.Linear(d_model, out_dim)# if out_dim != d_model else None

    def forward(self, x, context_indices, trg_indices): # [batch, seq_len, d_model], [batch, seq_len] # True will be ignored by the attention # https://pytorch.org/docs/stable/generated/torch.nn.Transformer.html
        x = self.embed(x) # [batch, seq_len, d_model] or [batch, num_context_toks, d_model]
        batch, seq, dim = x.shape
        # x = x * self.pos_enc(context_indices)
        # x = x + self.pos_emb[0,context_indices]

        # # pred_tokens = self.cls * self.pos_enc(trg_indices) # [M, num_trg_toks, d_model]
        # pred_tokens = self.cls + self.pos_emb[0,trg_indices]
        # # print("pred fwd", x.shape, pred_tokens.shape)
        # x = torch.cat([x, pred_tokens], dim=1) # [batch, seq_len+num_trg_toks, d_model]
        # out = self.transformer(x)

        x = torch.cat([x, self.cls.expand(*trg_indices.shape,-1)], dim=1) # [batch, num_ctx_toks+num_trg_toks, d_model]
        out = self.transformer(x, torch.cat([context_indices, trg_indices], dim=-1))

        out = self.norm(out)
        out = out[:,seq:] # [batch, num_trg_toks, d_model]
        out = self.lin(out)
        # print("pred fwd", out.shape)
        return out # [seq_len, batch_size, ntoken]


In [ ]:
# @title IJEPA
import torch
import torch.nn as nn
import torch.nn.functional as F
device = "cuda" if torch.cuda.is_available() else "cpu"

class IJEPA(nn.Module):
    def __init__(self, in_dim=3, d_model=64, out_dim=None, nlayers=1, n_heads=4):
        super().__init__()
        self.out_dim = out_dim = out_dim or d_model
        self.patch_size = 4 # 4
        self.student = ViT(self.patch_size, in_dim, d_model, out_dim=out_dim, n_heads=n_heads, nlayers=nlayers, drop=0.)
        self.predicter = TransformerPredictor(out_dim, d_model//2, out_dim, n_heads=4, nlayers=1, drop=0.)
        # self.predicter = TransformerPredictor(out_dim, 3*d_model//8, out_dim, n_heads=4, nlayers=1, drop=0.)
        import copy
        self.teacher = copy.deepcopy(self.student)
        self.teacher.requires_grad_(False)

    def loss(self, x): #
        b,c,h,w = x.shape
        # print('ijepa loss x',x.shape)
        hw=(8,8)
        # hw=(32,32)
        mask_collator = MaskCollator(hw, enc_mask_scale=(.85, 1.), pred_mask_scale=(.15, .2), aspect_ratio=(.75, 1.5), nenc=1, npred=4, min_keep=4, allow_overlap=False)
        collated_masks_enc, collated_masks_pred = mask_collator(b)
        context_indices, trg_indices = torch.stack(collated_masks_enc).squeeze(0), torch.stack(collated_masks_pred).transpose(0,1).flatten(1).unique(dim=1) # [num_msk, b,num_tok]->[b,num_tok] # [64, 65], [64, 32]
        # print(context_indices.shape, trg_indices.shape)

        # context_indices, trg_indices = simplexmask2d(hw, ctx_scale=(.85,1), trg_scale=(.7,.8), B=b, chaos=[3,.5])
        # context_indices, trg_indices = simplexmask2d(hw, ctx_scale=(.85,1), trg_scale=(.7,.8), B=b, chaos=[5,.5])
        # context_indices, trg_indices = simplexmask2d(hw, ctx_scale=(.05,.2), trg_scale=(.7,.8), B=b, chaos=[3,1])
        # context_indices, trg_indices = context_indices.repeat(b,1), trg_indices.repeat(b,1)

        # zero_mask = torch.zeros(b ,*hw, device=device).flatten(1)
        # zero_mask[torch.arange(b).unsqueeze(-1), context_indices] = 1
        # x_ = x * F.interpolate(zero_mask.reshape(b,1,*hw), size=x.shape[2:], mode='nearest-exact') # zero masked locations

        sx = self.student(x, context_indices=context_indices) # [batch, num_context_toks, out_dim]
        # print('ijepa loss sx',sx.shape)
        sy_ = self.predicter(sx, context_indices=context_indices, trg_indices=trg_indices) # [batch*M, num_trg_toks, out_dim]
        sy_ = F.layer_norm(sy_, (sy_.size(-1),))
        with torch.no_grad():
            sy = self.teacher(x.detach()) # [batch, num_trg_toks, out_dim]
            sy = sy[torch.arange(sy.shape[0]).unsqueeze(-1), trg_indices] # [batch, num_context_toks, d_model] # nan bec len(trg_ind)==0 # print('loss sy',torch.isnan(sy).any())
            sy = F.layer_norm(sy, (sy.size(-1),))
        loss = F.mse_loss(sy, sy_)
        return loss

    def forward(self, x): # [batch, T, 3]
        sx = self.student(x)
        out = sx.mean(dim=1)
        return out

# min_s=0.15, max_s, M
# trg.15.2M4 C.85 1
# dont normalise data
# randmsk < simplex 1msk < multiblk
# teacher=trans ~? teacher=copy
# learn convemb
# lr pred,student: 3e-3/1e-2, 1e-3

# lr 1e-3 < 1e-2,1e-3? slower but dun plateau

# vit 1lyr 15.4sec 15.6
# hiera downsampling attn 20.5

# ijepa = IJEPA(in_dim=3, d_model=64, out_dim=64, nlayers=4, n_heads=4).to(device)
ijepa = IJEPA(in_dim=3, d_model=64, out_dim=64, nlayers=1, n_heads=8).to(device)
# ijepa = IJEPA(in_dim=3, d_model=32, out_dim=32, nlayers=1, n_heads=4).to(device)
# optim = torch.optim.AdamW(ijepa.parameters(), lr=1e-3) # 1e-3?
optim = torch.optim.AdamW([{'params': ijepa.student.parameters()},
#     {'params': ijepa.predicter.parameters(), 'lr': 3e-3}], lr=1e-3, weight_decay=1e-2) # good 3e-3,1e-3 ; default 1e-2, 5e-2
    {'params': ijepa.predicter.parameters(), 'lr': 1e-2}], lr=1e-3, weight_decay=1e-2)

# https://github.com/facebookresearch/ijepa/blob/main/configs/in1k_vith14_ep300.yaml
# d_model 1024,384
# depth 12,6/12
# wd 5e-2 - 4e-1
# adamw 1e-4 - 1e-3 - 1e-6
# ema 0.996-1

print(sum(p.numel() for p in ijepa.parameters() if p.requires_grad)) # 27584
# print(sum(p.numel() for p in ijepa.parameters())) # 27584
# print(sum(p.numel() for p in ijepa.predicter.transformer_encoder.parameters() if p.requires_grad)) # 27584
# d_model^2 * nlayers

x = torch.rand((64,3,32,32), device=device)
out = ijepa.loss(x)
print(out.shape)

class Classifier(nn.Module):
    def __init__(self, in_dim, num_classes=10):
        super().__init__()
        self.classifier = nn.Linear(in_dim, num_classes)
    def forward(self, x): return self.classifier(x)
classifier = Classifier(ijepa.out_dim, 10).to(device)
coptim = torch.optim.SGD(classifier.parameters(), lr=1e-3)
# optim = torch.optim.AdamW([{'params': ijepa.parameters()}, {'params': classifier.parameters(), 'lr': 1e-3}], lr=1e-3)



In [ ]:
for name, param in ijepa.named_parameters():
    if param.requires_grad:
        print(name, param)

In [ ]:
# @title wandb
!pip install -q wandb
import wandb # https://docs.wandb.ai/quickstart
wandb.login(key='487a2109e55dce4e13fc70681781de9f50f27be7')
try: run.finish()
except NameError: pass
run = wandb.init(project="ijepa", config={"model": "res18",})

In [ ]:
# @title strain ctrain test
import torch
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
scaler = torch.GradScaler()
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

def strain(model, dataloader, optim, scheduler=None): # train function with automatic mixed precision
    model.train()
    for i, (x, _) in enumerate(dataloader):
        x = x.to(device)#.to(torch.bfloat16) # [b,c,h,w] -> [b,h*w,c]
        with torch.autocast(device_type=device, dtype=torch.bfloat16): # bfloat16 float16
            loss = model.loss(x)
        optim.zero_grad()
        scaler.scale(loss).backward()
        # scaler.unscale_(optim)
        # torch.nn.utils.clip_grad_norm_(model.parameters(), 5) # 0.5
        scaler.step(optim)
        scaler.update()

        with torch.no_grad():
            m=0.999 # 0.99 m = next(momentum_scheduler)
            norms=[]
            for param_q, param_k in zip(model.student.parameters(), model.teacher.parameters()):
                param_k.data.mul_(m).add_((1.-m) * param_q.detach().data)

        # if scheduler is not None: scheduler.step()
        if i%10==0: print("strain",loss.item())
        try: wandb.log({"loss": loss.item()})
        except NameError: pass

        if i>=50: break
    writer.close()

def ctrain(model, classifier, dataloader, coptim, scheduler=None): # train function with automatic mixed precision
    model.eval()
    classifier.train()
    for i, (x, y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device) # [batch, ]
        with torch.autocast(device_type=device, dtype=torch.bfloat16): # bfloat16 float16
            with torch.no_grad():
                sx = model(x).detach()
            y_ = classifier(sx)
            loss = F.cross_entropy(y_, y)
        coptim.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(coptim)
        scaler.update()
        print("classify",loss.item())
        try: wandb.log({"closs": loss.item()})
        except NameError: pass
        if i>=10: break


def test(model, classifier, dataloader):
    model.eval()
    classifier.eval()
    for i, (x, y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device) # [batch, ]
        with torch.no_grad():
            sx = model(x)
            y_ = classifier(sx)
        correct = (y==y_.argmax(dim=1)).sum().item()
        print(correct/len(y))
        try: wandb.log({"correct": correct/len(y)})
        # try: wandb.log({"correct": correct/len(y), "rankme": rankme, "lidar": lidar})
        except NameError: pass
        if i>=10: break

import time
start = begin = time.time()
# for i in range(1):
for i in range(1000): # 1000
    print(i)
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True)

    strain(ijepa, train_loader, optim)
    ctrain(ijepa, classifier, train_loader, coptim)
    test(ijepa, classifier, test_loader)

    print('time:',time.time() - start, (time.time()-begin)/(i+1))
    start = time.time()


In [ ]:
# @title supervised train test
import torch
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
scaler = torch.GradScaler()

# def strain(model, dataloader, optim, scheduler=None):
def strain(model, classifier, dataloader, optim, coptim, scheduler=None):
    model.train()
    classifier.train()
    for i, (x, y) in enumerate(dataloader):
        x = x.to(device)#.to(torch.bfloat16) # [b,c,h,w] -> [b,h*w,c]
        y = y.to(device)

        with torch.autocast(device_type=device, dtype=torch.bfloat16): # bfloat16 float16
            sx = model(x)
            y_ = classifier(sx)
            loss = F.cross_entropy(y_, y)

        optim.zero_grad()
        # print(loss)
        scaler.scale(loss).backward()

        # for p in list(filter(lambda p: p.grad is not None, model.parameters())):
        #     print(p.grad.data.norm(2).item())
        # print("max grad norm", max([p.grad.data.norm(2).item() for p in list(filter(lambda p: p.grad is not None, model.parameters()))]))
        # scaler.unscale_(optim)
        # torch.nn.utils.clip_grad_norm_(model.parameters(), 10) # 0.5

        scaler.step(optim)
        scaler.update()

        # if scheduler is not None: scheduler.step()
        print("strain",loss.item())
        # for param in ijepa.student.cls: print(param.data)
        # for param in ijepa.predicter.cls: print(param.data)
        try: wandb.log({"loss": loss.item()})
        except NameError: pass
        if i>=50: break


# def test(model, dataloader):
def test(model, classifier, dataloader):
    model.eval()
    classifier.eval()
    for i, (x, y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device) # [batch, ]
        # .to(torch.bfloat16)

        with torch.no_grad():
            sx = model(x)
            y_ = classifier(sx)
        loss = F.cross_entropy(y_, y)
        correct = (y==y_.argmax(dim=1)).sum().item()
        print(correct/len(y))
        try: wandb.log({"correct": correct/len(y), "closs": loss.item()})
        except NameError: pass
        if i>=10: break


for i in range(1000):
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)

    # np.random.shuffle(train_indices); np.random.shuffle(val_indices)
    # train_sampler, valid_sampler = SubsetRandomSampler(train_indices), SubsetRandomSampler(val_indices)
    # # batch_size = 64 #512
    # train_loader = DataLoader(train_data, sampler=train_sampler, pin_memory=True, batch_size=batch_size, num_workers=2, drop_last=True) # num_workers = 4
    # test_loader = DataLoader(train_data, sampler=valid_sampler, pin_memory=True, batch_size=batch_size, num_workers=2, drop_last=True)

    # strain(ijepa, train_loader, optim)
    strain(ijepa, classifier, train_loader, optim, coptim)
    test(ijepa, classifier, test_loader)

    # strain(violet, train_loader, voptim)
    # test(violet, test_loader)


In [ ]:
# @title save/load
from google.colab import drive
drive.mount('/content/drive')
folder='/content/drive/MyDrive/jepa/'

# # modelsd, optimsd = torch.load(folder+'SeqJEPA.pkl', map_location=device).values()
# modelsd, optimsd = torch.load('SeqJEPA.pkl', map_location=device).values()
# seq_jepa.load_state_dict(modelsd, strict=False)
# optim.load_state_dict(optimsd)

In [ ]:
checkpoint = {'model': seq_jepa.state_dict(), 'optimizer': optim.state_dict()}
torch.save(checkpoint, folder+'IJEPA.pkl')
# torch.save(checkpoint, 'IJEPA.pkl')

## trash

In [ ]:
# @title GoldenGateRoPE2d
import math
import torch
import torch.nn as nn
device = 'cuda' if torch.cuda.is_available() else 'cpu'

class GoldenGateRoPE2d(nn.Module): # jerryxio.ng/posts/nd-rope
    def __init__(self, image_size: tuple[int, int], n_heads, n_freqs):
        super().__init__()
        n_zero_freqs = 1
        min_freq, max_freq = .2, 20
        speed = torch.cat([torch.zeros(n_zero_freqs), min_freq * (max_freq/min_freq) ** torch.linspace(0,1,n_freqs-n_zero_freqs)]) # [n_freqs]
        direction_spacing = math.pi * (math.sqrt(5)-1)/2
        phi = torch.arange(n_heads * n_freqs).reshape(n_heads, n_freqs) * direction_spacing # [n_heads, n_freqs]
        directions = torch.stack((torch.cos(phi), torch.sin(phi)), dim=-1) # [n_heads, n_freqs, 2]
        # print('directions', directions)
        vel = speed.unsqueeze(-1) * directions # speed in direction[n_heads, n_freqs, 2]

        H, W = image_size
        xlim, ylim = math.sqrt(W / H), math.sqrt(H / W)
        print(xlim, ylim)
        y, x = torch.meshgrid(torch.linspace(-ylim, ylim, H), torch.linspace(-xlim, xlim, W), indexing="ij") # [h,w], y:row_num, x:col_num
        # y, x = torch.meshgrid(torch.arange(h), torch.arange(w), indexing="ij") # print(y,x) # [h,w], y:row_num, x:col_num

        pos = torch.stack((x, y), dim=-1).reshape(H, W, 1, 1, 2) # cartesian coords

        theta = (vel*pos).sum(dim=-1) # [h,w,n_heads,n_freqs,2]->[h,w,n_heads,d_head]
        print('theta', theta.shape)
        self.cos, self.sin = torch.cos(theta), torch.sin(theta)
        print('self.cos', self.cos.shape)

    def forward(self, input): # [b,h,w,n_head,d_head]
        # x, y = input.float().chunk(2, dim=-1) # [b,h,w,n_head,n_freqs]
        x, y = input.float().unflatten(-1, (-1,2)).chunk(2, dim=-1)
        x, y = x.squeeze(-1), y.squeeze(-1)
        x_out = x * self.cos - y * self.sin
        y_out = x * self.sin + y * self.cos
        # output = torch.cat((x_out, y_out), dim=-1)
        output = torch.stack((x_out, y_out), dim=-1).flatten(-2)
        return output.type_as(input)



class GoldenGateRoPE2d2(nn.Module): # jerryxio.ng/posts/nd-rope
    def __init__(self, image_size, n_heads, n_freqs):
        super().__init__()
        n_zero_freqs = 1
        min_freq, max_freq = .2, 20
        intv = math.pi * (math.sqrt(5)-1)/2
        speed = torch.cat([torch.zeros(n_zero_freqs), min_freq * (max_freq/min_freq) ** torch.linspace(0,1,n_freqs-n_zero_freqs)]) # [n_freqs]
        phi = torch.arange(n_heads*n_freqs).reshape(n_heads, n_freqs) * intv # [n_heads, n_freqs]
        direction = torch.stack((torch.cos(phi), torch.sin(phi)), dim=-1) # [n_heads, n_freqs, 2]
        vel = speed.unsqueeze(-1) * direction # speed in direction[n_heads, n_freqs, 2]
        h, w = image_size
        xlim, ylim = math.sqrt(w/h), math.sqrt(h/w)
        y, x = torch.meshgrid(torch.linspace(-ylim, ylim, h), torch.linspace(-xlim, xlim, w), indexing="ij") # [h,w], y:row_num, x:col_num
        pos = torch.stack([x, y], dim=-1)[...,None,None,:] # [h,w,1,1,2] cartesian coords
        theta = (vel*pos).sum(dim=-1) # [h,w,n_heads,n_freqs,2]->[h,w,n_heads,d_head]
        cos, sin = torch.cos(theta), torch.sin(theta)
        self.affine = torch.stack([cos, -sin, sin, cos], dim=-1).unflatten(-1, (2,2))

    def forward(self, input): # [b,h,w,n_head,d_head]
        return (self.affine @ input.unflatten(-1, (-1,2)).unsqueeze(-1)).flatten(-3)



image_size=(5,7)
n_heads=4
d_head=16
ggrope = GoldenGateRoPE2d(image_size, n_heads, d_head//2)
ggrope2 = GoldenGateRoPE2d2(image_size, n_heads, d_head//2)

# x = torch.rand(2, *image_size, n_heads, d_head)
out = ggrope(x)
out2 = ggrope2(x)
print(out.shape)
print(out2.shape)
# print(out[0])
# print(out2[0])
# print((out==out2)[0])
print((out==out2).all())



In [ ]:
%reload_ext tensorboard
%tensorboard --logdir runs